## Deploy a Model to ACI

In [ ]:
Deploying a model to ACI


## Connect to your workspace

To get started, connect to your workspace.

> **Note**: If you haven't already established an authenticated session with your Azure subscription, you'll be prompted to authenticate by clicking a link, entering an authentication code, and signing into Azure.

In [1]:
# import warnings
# warnings.filterwarnings('ignore')

import azureml.core
from azureml.core import Workspace, Model
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.37.0 to work with mm-machine-learning-devops-test


In [2]:
import os
directory_path = os.getcwd()
print("My current directory is : " + directory_path)
folder_name = os.path.basename(directory_path)
print("My directory name is : " + folder_name)

parent = os.path.dirname(directory_path)
parent_folder_name = os.path.basename(parent)
print("My user directory name is: " + parent_folder_name)

user = parent_folder_name
user = user.replace('_', '')
user = user.replace('-', '')
user = user[:10]
print('user=' + user)
experiment_name = parent_folder_name + '-003-code-a-thon-diabetes'

My current directory is : /mnt/batch/tasks/shared/LS_root/mounts/clusters/memasanz1/code/Users/memasanz/code-a-thon
My directory name is : code-a-thon
My user directory name is: memasanz
user=memasanz


In [3]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

diabetes_model version 7


In [4]:
import os

# Create a folder for the experiment files
experiment_folder = user + '-004-diabetes_code-a-thon-exp-deploy-aci'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

memasanz-004-diabetes_code-a-thon-exp-deploy-aci folder created


In [5]:
%%writefile $experiment_folder/score.py

import json
import joblib
import numpy as np
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'diabetes_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    
    # Return the predictions as JSON
    log_txt = 'Data:' + str(data) + ' - Predictions:' + str(predictions)
    print(log_txt)
    
    return json.dumps(predicted_classes)

Writing memasanz-004-diabetes_code-a-thon-exp-deploy-aci/score.py


In [6]:
%%writefile $experiment_folder/experiment_env.yml
name: experiment_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow

Writing memasanz-004-diabetes_code-a-thon-exp-deploy-aci/experiment_env.yml


In [7]:
from azureml.core import Environment

# Create a Python environment for the experiment (from a .yml file)
experiment_env = Environment.from_conda_specification(user + "-experiment-env", experiment_folder + "/experiment_env.yml")

# Let Azure ML manage dependencies
experiment_env.python.user_managed_dependencies = False 

# Print the environment details
print(experiment_env.name, 'defined.')
print(experiment_env.python.conda_dependencies.serialize_to_string())

memasanz-experiment-env defined.
name: experiment_env
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2
- scikit-learn
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults
  - pyarrow



In [13]:
inference_config = InferenceConfig(source_directory=experiment_folder,
                                   entry_script='score.py',
                                   environment=experiment_env)

# Configure the web service container
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, enable_app_insights=True)

# Deploy the model as a service
print('Deploying model...')
service_name = "diabetes-service"
service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)
print(service.state)

Deploying model...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-03-07 18:58:21+00:00 Creating Container Registry if not exists.
2022-03-07 18:58:21+00:00 Registering the environment.
2022-03-07 18:58:22+00:00 Use the existing image.
2022-03-07 18:58:22+00:00 Generating deployment configuration.
2022-03-07 18:58:23+00:00 Submitting deployment to compute.
2022-03-07 18:58:25+00:00 Checking the status of deployment diabetes-service..
2022-03-07 19:01:58+00:00 Checking the status of inference endpoint diabetes-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [14]:
print(service.get_logs())

2022-03-07T19:00:44,671315700+00:00 - iot-server/run 
2022-03-07T19:00:44,724578400+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-03-07T19:00:44,739650400+00:00 - rsyslog/run 
2022-03-07T19:00:44,750320800+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_0c5a9aa2def4b3c2501c1f40287a356b/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-03-07T19:00:45,135381600+00:00 - iot-server/finish 1 0
2022-03-07T19:00:45,141328200+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (69)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 97
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2022-03-07 19:00:46,530 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. 

In [15]:
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

Patient: [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22]
not-diabetic


You can send multiple observations to the service and get back a perdiction for each one

In [16]:
import json

# This time our input is an array of two feature arrays
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Patient [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] not-diabetic
Patient [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] not-diabetic


In [17]:
endpoint = service.scoring_uri
print(endpoint)

http://d45eccc9-e31e-487d-9edc-06ff2b344857.eastus.azurecontainer.io/score


In [18]:
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Patient [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] not-diabetic
Patient [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] not-diabetic


In [ ]:
service.delete()
print ('Service deleted.')